### Student name : Yogesh Haresh Bojja
### Student number : s3789918
### Course project number : Group 66

We have imported pandas, numpy, sklearn as we need to work with it. In cell[2] we have imported the data and displayed it in cell[3].

In [1]:
# Import library
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn import preprocessing

In [2]:
# Read csv file
diamonds = pd.read_csv("THA_diamonds.csv")

In [3]:
# display table
diamonds.head(5)

,cut,color,depth,price,carat
0,Good,D,63.6,low,0.44
1,Fair,F,64.2,low,0.45
2,Good,I,60.4,low,0.50
3,Good,F,56.8,low,0.45
4,Fair,F,64.3,low,0.45


### Part A.

We need to apply one-hot-encoding to categorical descriptive attributes('cut', 'color' & 'price') and scaling to numeric descriptive attributes('depth'). We have used 'get_dummies()' of pandas for one-hot-encoding. As 'cut' attribute has only two levels('Good'/'Fair') we will set 'drop_first' to TRUE. Thus, 'Good' is encoded as 1 and 'Fair' is encoded as 0. 'Color' and 'price' have more than two levels, therefore we will keep 'drop_first' to FALSE which will make seperate column for each level. For scaling we have defined a function 'min_max_scaler()' which will return the Min-Max-Scaling value. With the help of 'apply()' we will apply 'min_max_scaler()' function to all the values in 'depth' column and scale it.

In [4]:
# one-hot-encoding and scaling of the attributes
diamonds['cut'] = pd.get_dummies(diamonds['cut'], drop_first=True)

diamonds = pd.get_dummies(diamonds, columns=['color', 'price'])

min_val = min(diamonds['depth'])
max_val = max(diamonds['depth'])
def min_max_scaler(val):
    return(round((val-min_val)/(max_val-min_val), 4))
diamonds['depth'] = diamonds['depth'].apply(min_max_scaler)
    

After one-hot-encoding and scaling our dataset looks as follows:

In [5]:
# display table after one-hot-encoding and scaling
diamonds.tail(10)

,cut,depth,carat,color_D,color_F,color_I,price_high,price_low,price_medium,price_premium
202,1,0.1343,1.01,0,0,1,0,0,0,1
203,1,0.2015,0.91,0,0,1,0,0,0,1
204,0,0.7761,0.90,0,1,0,0,0,0,1
205,1,0.3209,0.92,0,0,1,0,0,0,1
206,0,0.8582,0.91,0,1,0,0,0,0,1
207,1,0.6269,0.96,0,1,0,0,0,0,1
208,0,0.1642,0.90,1,0,0,0,0,0,1
209,0,0.7015,0.90,0,1,0,0,0,0,1
210,1,0.2164,0.93,0,0,1,0,0,0,1
211,0,0.2687,0.90,0,1,0,0,0,0,1


### Part B.

We will be applying manual calculations of KNeighbors algorithm in this part. After training the model we need to predict value of target feature i.e. 'carat' for data = (cut : good, color : D, depth : 60, price : premium). Hence, we need to one-hot-encode and scale the prediction data. Referring to the above encoding we have encoded and scaled the attributes of data as shown below. DataFrame 'query' contains the data for which we need to predict the value of 'carat'.

In [6]:
# one-hot-encoding and scaling of the data for which we need to predict the value of 'carat'
data = {'cut':[1], 'depth':[60], 'color_D':[1], 'color_F':[0], 'color_I':[0], 'price_high':[0], 'price_low':[0], 
        'price_medium':[0], 'price_premium':[1]}

# Converting the prediction data to 'query' dataframe
query = pd.DataFrame(data)
    
def min_max_scaler(val):
    return(round((val-min_val)/(max_val-min_val), 5))
query['depth'] = query['depth'].apply(min_max_scaler)
    
query

,cut,depth,color_D,color_F,color_I,price_high,price_low,price_medium,price_premium
0,1,0.35075,1,0,0,0,0,0,1


For manual calculations of KNeighbors we have defined a function 'KNN_Algorithm()'. 'KNN_Algorithm()' accepts value of 'K' and boolean value for 'display_table' inorder to display table. We need to predict 'carat' for 'query'. Thus, we will find distance between each point in the dataframe and the 'query'. Distance of each row with respect to 'query' is stored in the new column 'distance'. Once we find all the distances, we will be sorting the dataframe in ascending order based on the column 'distance'. 'K' is the number of neighbors we are considering. We will calculate average of 'carat' for 'K' neighbors and return it as the prediction for 'query'. 'KNN_Algorithm()' also displays the distance table for our reference.

In [17]:
def KNN_Algorithm(k, display_table):
    # Create empty table 'data' with all the features and additional distance column.
    data = {'cut':[], 'depth':[], 'color_D':[], 'color_F':[], 'color_I':[], 'price_high':[], 'price_low':[], 
        'price_medium':[], 'price_premium':[], 'carat':[], 'distance':[]}
    
    # Calculating the distance between query and each row of the dataset.
    for x in diamonds.values:
        dist = np.power(np.sum([
            np.power( x[0] - query['cut'], 2),
            np.power( x[3] - query['color_D'],2),
            np.power( x[4] - query['color_F'],2),
            np.power( x[5] - query['color_I'],2),
            np.power( x[1] - query['depth'],2),
            np.power( x[6] - query['price_high'],2),
            np.power( x[7] - query['price_low'],2),
            np.power( x[8] - query['price_medium'],2),
            np.power( x[9] - query['price_premium'],2),
        ]), 1/2)
        # Storing the values of feature and distance in new table 'data'.
        data['cut'].append(x[0])
        data['color_D'].append(x[3])
        data['color_F'].append(x[4])
        data['color_I'].append(x[5])
        data['depth'].append(x[1])
        data['price_high'].append(x[6])
        data['price_low'].append(x[7])
        data['price_medium'].append(x[8])
        data['price_premium'].append(x[9])
        data['carat'].append(x[2])
        data['distance'].append(dist)

    # Convert table 'data' to pandas dataframe.
    data = pd.DataFrame(data)
    # Sort dataframe in ascending order by 'distance' column.
    data = data.sort_values('distance')
    # Display new table 'data' for our reference.
    if display_table == True:
        print("---- Distance Table ----")
        display(data)
    
    # Calculate sum of 'carat' for K neighbors.
    total = 0
    for i in range(k):
        total += data.iloc[i,9]
    
    # Return average of 'carat' as the prediction value for queried data 'query'. 
    return total/k


For 'K'=1, the distance table and prediction is as belows. Prediction is average of 'carat' for 1 neighbor i.e. 0.70/1 = 0.70.

In [8]:
manual_k_1 = KNN_Algorithm(k=1, display_table=True)
print("Manual calculation of carat for K=1 :",manual_k_1)

---- Distance Table ----


,cut,depth,color_D,color_F,color_I,price_high,price_low,price_medium,price_premium,carat,distance
198,1.0,0.3433,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.70,0.007450
200,1.0,0.1940,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.90,0.156750
208,0.0,0.1642,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.90,1.017252
28,1.0,0.3582,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.51,1.414233
205,1.0,0.3209,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.92,1.414529
10,1.0,0.3955,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.44,1.414921
67,1.0,0.4179,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.54,1.415807
114,1.0,0.2687,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.62,1.416592
59,1.0,0.2313,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.53,1.419249
95,1.0,0.2239,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.57,1.419891


Manual calculation of carat for K=1 : 0.7


### Part C.

For 'K'=5, the distance table and prediction is as belows. Prediction is average of 'carat' for 5 neighbor i.e. (0.70+0.90+0.90+0.51+0.92)/5=0.786.

In [9]:
manual_k_5 = KNN_Algorithm(k=5, display_table=True)
print("Manual calculation of carat for K=5 :",manual_k_5)

---- Distance Table ----


,cut,depth,color_D,color_F,color_I,price_high,price_low,price_medium,price_premium,carat,distance
198,1.0,0.3433,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.70,0.007450
200,1.0,0.1940,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.90,0.156750
208,0.0,0.1642,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.90,1.017252
28,1.0,0.3582,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.51,1.414233
205,1.0,0.3209,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.92,1.414529
10,1.0,0.3955,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.44,1.414921
67,1.0,0.4179,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.54,1.415807
114,1.0,0.2687,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.62,1.416592
59,1.0,0.2313,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.53,1.419249
95,1.0,0.2239,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.57,1.419891


Manual calculation of carat for K=5 : 0.7859999999999999


### Part D.

For 'K'=10, the distance table and prediction is as belows. Prediction is average of 'carat' for 10 neighbor i.e. (0.70+0.90+0.90+0.51+0.92+0.44+0.54+0.62+0.53+0.57)/10=0.663.

In [10]:
manual_k_10 = KNN_Algorithm(k=10, display_table=True)
print("Manual calculation of carat for K=10 :",manual_k_10)

---- Distance Table ----


,cut,depth,color_D,color_F,color_I,price_high,price_low,price_medium,price_premium,carat,distance
198,1.0,0.3433,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.70,0.007450
200,1.0,0.1940,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.90,0.156750
208,0.0,0.1642,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.90,1.017252
28,1.0,0.3582,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.51,1.414233
205,1.0,0.3209,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.92,1.414529
10,1.0,0.3955,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.44,1.414921
67,1.0,0.4179,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.54,1.415807
114,1.0,0.2687,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.62,1.416592
59,1.0,0.2313,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.53,1.419249
95,1.0,0.2239,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.57,1.419891


Manual calculation of carat for K=10 : 0.663


### Part E.

In this part we are going to verify our manual calculations by the output of 'KNeighborsRegressor()' from SKLearn. 

In [11]:
# p=2 for euclidean's distance
knn_regressor = KNeighborsRegressor(n_neighbors=1, p=2)
desc = diamonds.drop(columns = 'carat').values
target = diamonds['carat'].values
knn_regressor.fit(desc, target)

# Store value of prediction for k=1 in sklearn_k_1 for populating it in the table in Part E.
sklearn_k_1 = knn_regressor.predict(query)
print("SKlearn calculation of carat for K=1 :",sklearn_k_1[0])

SKlearn calculation of carat for K=1 : 0.7


In [12]:
# p=2 for euclidean's distance
knn_regressor = KNeighborsRegressor(n_neighbors=5, p=2)
desc = diamonds.drop(columns = 'carat').values
target = diamonds['carat'].values
knn_regressor.fit(desc, target)

# Store value of prediction for k=5 in sklearn_k_5 for populating it in the table in Part E.
sklearn_k_5 = knn_regressor.predict(query)
print("SKlearn calculation of carat for K=5 :",sklearn_k_5[0])

SKlearn calculation of carat for K=5 : 0.7859999999999999


In [13]:
# p=2 for euclidean's distance
knn_regressor = KNeighborsRegressor(n_neighbors=10, p=2)
desc = diamonds.drop(columns = 'carat').values
target = diamonds['carat'].values
knn_regressor.fit(desc, target)

# Store value of prediction for k=10 in sklearn_k_10 for populating it in the table in Part E.
sklearn_k_10 = knn_regressor.predict(query)
print("SKlearn calculation of carat for K=10 :",sklearn_k_10[0])

SKlearn calculation of carat for K=10 : 0.6630000000000001


After calculating the values by KNeighborsRegressor() we can conclude that results we get by manual calculations exactly matches with the results of KNeighborsRegressor().

### Part F.

Summary of the manual calculations is stored in table 'df_summary_manual' and summary of the results we got from KNeighborsRegressor() is stored in table 'df_summary_sklearn'. Results for manual calculation are fetched by executing 'KNN_Algorithm()' for respective values ok 'K'.

In [14]:
df_summary_manual = pd.DataFrame(columns=['method', 'prediction', 'is_best'])
df_summary_manual.loc[len(df_summary_manual)] = ["K=1", round(KNN_Algorithm(k=1, display_table=False), 3) , 1]
df_summary_manual.loc[len(df_summary_manual)] = ["K=5", round(KNN_Algorithm(k=5, display_table=False), 3) , 0]
df_summary_manual.loc[len(df_summary_manual)] = ["K=10", round(KNN_Algorithm(k=10, display_table=False), 3) , 0]

df_summary_manual

,method,prediction,is_best
0,K=1,0.700,1
1,K=5,0.786,0
2,K=10,0.663,0


Results we got from KNeighborsRegressor() is stored in variable 'sklearn_k_1', 'sklearn_k_5' and 'sklearn_k_10'.

In [15]:
df_summary_sklearn = pd.DataFrame(columns=['method', 'prediction', 'is_best'])
df_summary_sklearn.loc[len(df_summary_sklearn)] = ["K=1", round(sklearn_k_1[0], 3) , 1]
df_summary_sklearn.loc[len(df_summary_sklearn)] = ["K=5", round(sklearn_k_5[0], 3) , 0]
df_summary_sklearn.loc[len(df_summary_sklearn)] = ["K=10", round(sklearn_k_10[0], 3) , 0]

df_summary_sklearn

,method,prediction,is_best
0,K=1,0.700,1
1,K=5,0.786,0
2,K=10,0.663,0


Prediction value for 'query' was told to us as 0.71, and as this value closely matches for the results of k=1 we have set this value as the best in 'is_best' column for above 2 tables. 

#### Conclusion: 
Results from KNeighborsRegressor() and manual calculations match perfectly.